In [1]:
import numpy as np
import pandas as pd
import pickle, time
import tqdm, os

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cityname = 'TKY' # TKY

# NYC: 38333, 251, 425, 589
# TKY: 61858, 247, 358, 443

timezone = {'NYC':-5, 'TKY':9}[cityname]

data = pd.read_csv('datasets/dataset_TSMC2014_{}.csv'.format(cityname))

data['utcTimestamp'] = data['utcTimestamp'].apply(lambda x:time.mktime(time.strptime(x[:20] + x[-4:])))
data['utcTimestamp'] = data['utcTimestamp'].apply(lambda x: (x//(60*60)+timezone)//(24))
data['latitude'] = data['latitude'].round(2)
data['longitude'] = -data['longitude'].round(2)

data['venueId'], venue_list = data['venueId'].factorize(sort=True)
#for iter in range(0):
#    item_threshhold = np.bincount(data.iloc[:, 1])[np.bincount(data.iloc[:, 1]).argsort()[-1000]]
#    item_remained_index = (np.bincount(data.iloc[:, 1])>=item_threshhold)
#    data = data[data.iloc[:, 1].apply(lambda x:item_remained_index[x])]
#    
#    user_threshhold = 20
#    user_remained_index = (np.bincount(data.iloc[:, 0])>=user_threshhold)
#    data = data[data.iloc[:, 0].apply(lambda x:user_remained_index[x])]

data['venueId'], user_list = data['venueId'].factorize(sort=True)
data['latitude'], lat_list = data['latitude'].factorize(sort=True)
data['longitude'], lon_list = data['longitude'].factorize(sort=True)
data['venueCategory'], venue_cate_list = data['venueCategory'].factorize(sort=True)

In [14]:
for iter in range(1):
    item_threshhold = 0
    item_remained_index = (np.bincount(data['venueId'])>=item_threshhold)
    data = data[data['venueId'].apply(lambda x:item_remained_index[x])]

    user_threshhold = 0
    user_remained_index = (np.bincount(data['userId'])>=user_threshhold)
    data = data[data['userId'].apply(lambda x:user_remained_index[x])]

    data['userId'], user_list = data['userId'].factorize(sort=True)
    data['venueId'], item_list = data['venueId'].factorize(sort=True)

print('Number Users : {}'.format(len(user_list)))
print('Number Items : {}'.format(len(item_list)))

Number Users : 1083
Number Items : 5135


In [15]:
user_dict = dict([(user_id, []) for user_id in data.iloc[:,0].unique()])
distribution = np.zeros([500])
start = []
number = []
length = []

for row_id in tqdm.tqdm(range(data.shape[0])):
    user = data.iloc[row_id, 0]
    item = data.iloc[row_id, 1]
    cate = data.iloc[row_id, 3]
    lat = data.iloc[row_id, 4]
    lon = data.iloc[row_id, 5]
    t = data.iloc[row_id, 7]
    user_dict[user].append([t, item, cate, lat, lon])
    
for user in tqdm.tqdm(user_dict):
    user_dict[user]=np.array(user_dict[user])
    new_time = pd.factorize(user_dict[user][:, 0], sort=True)[0]
    user_dict[user] = np.concatenate([new_time[:, np.newaxis], user_dict[user]], axis=-1)
    number.append(user_dict[user].shape[0])
    length.append(user_dict[user][:,0].max())
    
user_dict2 = {}
for iter, key in tqdm.tqdm(enumerate(user_dict)):
    if length[iter] >= 28:
        user_dict2[key] = user_dict[key]
    
with open('{0}_dataset.pkl'.format(cityname), 'wb') as f:
    pickle.dump(user_dict2, f)
print([len(user_list), len(venue_cate_list), len(lat_list), len(lon_list)])

100%|██████████| 1083/1083 [00:00<00:00, 1158.25it/s]
1083it [00:00, 419082.13it/s]

[1083, 251, 45, 60]
